# Graphiti + LangGraph + Lapa LLM Demo

This notebook demonstrates an AI agent with long-term memory using:
- **Lapa LLM** - Ukrainian language model via hosted Lapathon API
- **Graphiti** - Temporal knowledge graph for memory
- **LangGraph** - Agent orchestration
- **Neo4j** - Graph database storage

## 1. Setup and Imports

## 1.5. Test Graphiti Message Alternation Fix

Test that our wrapper correctly handles message role alternation issues

In [1]:
import asyncio
import logging
from datetime import datetime
import sys

# Add project root to path
sys.path.insert(0, '../')
# Enable detailed logging

In [2]:
import asyncio
import logging
from datetime import datetime
from langchain_core.messages import HumanMessage

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Import our modules
from config.settings import settings
from clients.llm_client import get_llm_client
from agent.graph import get_agent_app
from agent.state import create_initial_state
from utils.langsmith_setup import setup_langsmith

# Ініціалізація LangSmith
setup_langsmith()
print("✅ Imports successful")

INFO:agent.storage.message_store:Message store initialized at /tmp/messages.db


✅ Imports successful


## 2. Check Neo4j Status

Verify that Neo4j is running before starting the demo

✅ Neo4j is running


True

## 3. Initialize Clients

In [3]:
# Initialize LLM client
llm_client = get_llm_client()
print(f"✅ LLM Client initialized: {llm_client.model_name}")

# Initialize Graphiti client
# graphiti_client = await get_graphiti_client()
# await graphiti_client.initialize()
# print("✅ Graphiti Client initialized", graphiti_client._initialized)

# Get agent app
agent = get_agent_app()
print("✅ Agent Graph compiled")

INFO:agent.graph:Initializing global agent application
INFO:agent.graph:Creating improved agent graph with conflict resolution chain...
INFO:agent.graph:Improved agent graph compiled successfully with conflict resolution


✅ LLM Client initialized: lapa
✅ Agent Graph compiled


## 4. Test LLM Connection

Let's verify that our LLM is working and responds in Ukrainian

In [6]:
test_messages = [
    {"role": "system", "content": "Ти - корисний AI асистент."},
    {"role": "user", "content": "Привіт! Як справи?"}
]

response = await llm_client.generate_async(test_messages)
print("LLM Response:")
print(response)

INFO:openai._base_client:Retrying request to /chat/completions in 0.379681 seconds
INFO:openai._base_client:Retrying request to /chat/completions in 0.801287 seconds
ERROR:clients.llm_client:Error generating response: Connection error.


APIConnectionError: Connection error.

In [ ]:
# Get graph statistics
stats = await graphiti_client.get_graph_stats()
print(f"📊 Graph Memory Stats:")
print(f"   Nodes: {stats['node_count']}")
print(f"   Relationships: {stats['relationship_count']}")
print(f"\n💡 The agent is learning and building a knowledge graph!")

In [ ]:
# Get graph statistics
stats = await graphiti_client.get_graph_stats()
print(f"📊 Graph Stats:")
print(f"   Nodes: {stats['node_count']}")
print(f"   Relationships: {stats['relationship_count']}")

# Search for specific information
search_results = await graphiti_client.search("Олександр Київ")
print(f"\n🔍 Search results for 'Олександр Київ': {len(search_results)} found")
for i, search_item in enumerate(search_results[:3], 1):
    print(f"   {i}. {search_item.get('content', 'N/A')[:100]}...")

## 7. Second Conversation: Testing Memory Recall

Now let's ask a question that requires recalling information from previous conversation

## 8. Third Conversation: More Complex Query

## 9. Visualize Knowledge Graph

Let's query Neo4j directly to see what entities and relationships were created

## Summary

### What We Demonstrated:
1. ✅ **Hosted Lapa LLM** - Ukrainian language model via Lapathon API
2. ✅ **Hosted Qwen Embeddings** - Semantic search using hosted embeddings
3. ✅ **Graphiti Memory** - Temporal knowledge graph for long-term memory
4. ✅ **LangGraph Agent** - Three-node pipeline (retrieve → generate → save)
5. ✅ **Memory Recall** - Context-aware responses using graph memory

### Architecture:
- **LLM**: Lapa model @ http://146.59.127.106:4000
- **Embeddings**: text-embedding-qwen (hosted)
- **Memory**: Graphiti + Neo4j graph database
- **Agent**: LangGraph with persistent state

### Next Steps:
1. Explore Neo4j Browser: http://localhost:7474
2. Try different conversation topics
3. Test memory across multiple sessions
4. Experiment with Mamay model (change VLLM_MODEL_NAME=mamay in .env)

## 10. Summary and Next Steps

### What We Demonstrated:
1. ✅ Lapa LLM integration via vLLM with structured outputs
2. ✅ Graphiti knowledge graph for long-term memory
3. ✅ LangGraph agent orchestration with state management
4. ✅ Memory retrieval and contextual responses
5. ✅ Graph visualization and querying

### Key Features:
- **Temporal Memory**: Graphiti tracks when information was learned
- **Semantic Search**: Hybrid search (embeddings + BM25 + graph traversal)
- **Context Awareness**: Agent uses retrieved memories to personalize responses
- **Ukrainian Support**: Lapa LLM optimized for Ukrainian language

### Next Steps:
1. Add more conversations to build richer memory
2. Experiment with different query types
3. Visualize graph in Neo4j Browser (http://localhost:7474)
4. Test with multiple users/sessions
5. Implement memory cleanup strategies for old data

## 11. Cleanup (Optional)

In [ ]:
# Uncomment to clear all graph data
# from neo4j import AsyncGraphDatabase
#
# async def clear_graph():
#     driver = AsyncGraphDatabase.driver(
#         settings.neo4j_uri,
#         auth=(settings.neo4j_user, settings.neo4j_password)
#     )
#     async with driver.session(database=settings.neo4j_database) as session:
#         await session.run("MATCH (n) DETACH DELETE n")
#     await driver.close()
#     print("✅ Graph cleared")
#
# await clear_graph()